In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="OsDkWz30d9L1pmUgqIcW")
project = rf.workspace("roboflow-universe-projects").project("license-plate-recognition-rxg4e")
version = project.version(4)
dataset = version.download("yolov5")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to License-Plate-Recognition-4 in yolov5pytorch:: 100%|██████████| 48488/48488 [00:02<00:00, 17842.34it/s]
